In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [2]:
# Load detectors.csv
detectors = pd.read_csv('dataset/detectors.csv')
detectors['detid'] = detectors['detid'].astype('str')
detectors.set_index(['citycode', 'detid'], inplace=True)

# Pull and clean up traffic data for a city
def convert_traffic_data(city):
    # Load {city}.csv
    city_data = pd.read_csv(f'dataset/cities/{city}.csv')

    # Divide the interval column by 3600 to get the hour
    city_data['interval'] = city_data['interval'].apply(lambda x: (x // 3600) % 24)

    # Merge rows
    city_data = city_data.groupby(['city', 'detid', 'day', 'interval']).mean().reset_index()

    # Convert day column to day of the week
    city_data['date'] = city_data['day']
    city_data['day'] = pd.to_datetime(city_data['day']).dt.dayofweek

    # Drop the datetime and timestamp columns
    city_data = city_data.drop(columns=['error', 'speed'])
    # Merge with detectors.csv
    city_data['detid'] = city_data['detid'].astype('str')
    city_data = city_data.merge(detectors, left_on=['city', 'detid'], right_index=True)

    # Remove rows with missing data
    city_data = city_data.drop(columns=['road', 'long', 'lat', 'detid'])

    city_data['linkid'] = city_data['linkid'].astype('str')
    city_data['limit'] = city_data['limit'].astype('int64', errors='ignore').fillna(0)
    city_data['lanes'] = city_data['lanes'].astype('int64', errors='ignore').fillna(1)

    # Convert the fclass column to a range
    fclass_ids = [ 'other', 'living_street', 'residential', 'service', 'tertiary_link', 'tertiary', 'secondary_link', 'secondary', 'primary_link', 'primary', 'trunk_link', 'trunk', 'motorway_link', 'motorway' ]
    city_data['fclass'] = city_data['fclass'].apply(lambda x: fclass_ids.index(x))

    # Multiply the flow column by the lane count and the road length
    city_data['flow'] = city_data['flow'] * city_data['lanes'] * city_data['length']

    # Convert the limit column to int
    city_data['limit'] = city_data['limit'].astype('int64', errors='ignore').fillna(0)

    # Average the columns over all links in a city
    city_data_avg = city_data.copy()
    city_data_avg = city_data_avg.drop(columns=['linkid', 'date'])
    city_data_avg = city_data_avg.groupby(['city', 'day', 'interval']).mean().reset_index()

    return city_data, city_data_avg

cities = [ 'augsburg', 'basel', 'bern', 'bolton', 'bordeaux', 'bremen', 'cagliari', 'constance', 'darmstadt', 'essen', 'graz', 'groningen', 'hamburg', 'kassel', 'london', 'luzern', 'manchester', 'marseille', 'munich', 'paris', 'rotterdam', 'santander', 'speyer', 'strasbourg', 'stuttgart', 'torino', 'toulouse', 'vilnius', 'wolfsburg', 'zurich' ]
# cities = ['madrid']

data = pd.DataFrame()
data_avg = pd.DataFrame()

for city in cities:
    print(f'Loading {city}...          ', end='\r')
    city_data, city_data_avg = convert_traffic_data(city)

    data = pd.concat([data, city_data])
    data_avg = pd.concat([data_avg, city_data_avg])

/var/folders/9j/ddzkzm7x2bv5txk3k7qblr_80000gn/T/ipykernel_75504/1678766570.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  city_data = pd.read_csv(f'dataset/cities/{city}.csv')


In [3]:
data.head()

,city,day,interval,flow,occ,date,length,pos,fclass,limit,lanes,linkid
0,augsburg,5,0,1.568041,0.0025,2017-05-06,0.112003,0.003423,5,30,1.0,737.0
1,augsburg,5,1,0.448012,0.0000,2017-05-06,0.112003,0.003423,5,30,1.0,737.0
2,augsburg,5,2,0.784021,0.0000,2017-05-06,0.112003,0.003423,5,30,1.0,737.0
3,augsburg,5,3,0.672018,0.0000,2017-05-06,0.112003,0.003423,5,30,1.0,737.0
4,augsburg,5,4,0.560015,0.0000,2017-05-06,0.112003,0.003423,5,30,1.0,737.0


In [4]:
data_avg.head()

,city,day,interval,flow,occ,length,pos,fclass,limit,lanes
0,augsburg,0,0,7.257053,0.054564,0.20692,0.035397,5.656525,47.302231,1.0
1,augsburg,0,1,3.309013,0.049596,0.20692,0.035397,5.656525,47.302231,1.0
2,augsburg,0,2,2.403672,0.048928,0.20692,0.035397,5.656525,47.302231,1.0
3,augsburg,0,3,2.903688,0.049114,0.20692,0.035397,5.656525,47.302231,1.0
4,augsburg,0,4,6.994569,0.053670,0.20692,0.035397,5.656525,47.302231,1.0


In [5]:
# Print row counts for each city
data_avg.groupby('city').count()

,day,interval,flow,occ,length,pos,fclass,limit,lanes
city,,,,,,,,,
augsburg,168,168,168,168,168,168,168,168,168
basel,168,168,168,168,168,168,168,168,168
bern,168,168,168,168,168,168,168,168,168
bolton,125,125,125,125,125,125,125,125,125
bordeaux,168,168,168,168,168,168,168,168,168
bremen,168,168,168,168,168,168,168,168,168
cagliari,120,120,120,120,120,120,120,120,120
constance,168,168,168,168,168,168,168,168,168
darmstadt,120,120,120,120,120,120,120,120,120


In [6]:
cities = [ 'augsburg', 'basel', 'bern', 'bordeaux', 'bremen', 'constance', 'essen', 'hamburg', 'london', 'luzern', 'manchester', 'marseille', 'paris', 'speyer', 'strasbourg', 'torino', 'wolfsburg', 'zurich' ]

data = data[data['city'].isin(cities)]
data_avg = data_avg[data_avg['city'].isin(cities)]

In [7]:
# Cluster the cities
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=100)

X = []
for city in cities:
    X.append(data_avg[data_avg['city'] == city].drop(columns=['city', 'day', 'interval']).replace([np.inf, -np.inf], np.nan).fillna(0).values.flatten())
X = np.array(X)
y = np.array(cities)

kmeans.fit(X)

# Print labels
for i in range(len(cities)):
    print(f'{cities[i]}: {kmeans.labels_[i]}')

augsburg: 2
basel: 2
bern: 2
bordeaux: 3
bremen: 2
constance: 2
essen: 0
hamburg: 2
london: 2
luzern: 0
manchester: 4
marseille: 3
paris: 1
speyer: 2
strasbourg: 0
torino: 4
wolfsburg: 0
zurich: 2


In [8]:
# Add hour sin/cos and day sin/cos columns
data['hour_sin'] = np.sin(2 * np.pi * data['interval'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['interval'] / 24)
data['day_sin'] = np.sin(2 * np.pi * data['day'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day'] / 7)

# Remove days without all 24 hours
# data = data[data.groupby(['city', 'day'])['interval'].transform('count') == 24]

In [9]:
# Index by city and linkid
data = data.reset_index().set_index(['city', 'linkid', 'date', 'interval'])

print(data.shape)
data.head()

(8252987, 13)


index  day      flow     occ    length  \
city     linkid date       interval                                           
augsburg 737.0  2017-05-06 0             0    5  1.568041  0.0025  0.112003   
                           1             1    5  0.448012  0.0000  0.112003   
                           2             2    5  0.784021  0.0000  0.112003   
                           3             3    5  0.672018  0.0000  0.112003   
                           4             4    5  0.560015  0.0000  0.112003   

                                          pos  fclass  limit  lanes  hour_sin  \
city     linkid date       interval                                             
augsburg 737.0  2017-05-06 0         0.003423       5     30    1.0  0.000000   
                           1         0.003423       5     30    1.0  0.258819   
                           2         0.003423       5     30    1.0  0.500000   
                           3         0.003423       5     30    1.0  0.707107   
                           4         0.003423       5     30    1.0  0.866025   

                                     hour_cos   day_sin   day_cos  
city     linkid date       interval                                
augsburg 737.0  2017-05-06 0         1.000000 -0.974928 -0.222521  
                           1         0.965926 -0.974928 -0.222521  
                           2         0.866025 -0.974928 -0.222521  
                           3         0.707107 -0.974928 -0.222521  
                           4         0.500000 -0.974928 -0.222521

In [11]:
# Create a train/test/validation split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=False)
train, val = train_test_split(train, test_size=0.2, shuffle=False)

print(train.shape, test.shape, val.shape)

(5281911, 13) (1650598, 13) (1320478, 13)


In [12]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train, val_df=val, test_df=test,
               label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                            enumerate(label_columns)}
            self.column_indices = {name: i for i, name in
                                enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    def plot(self, model=None, plot_col='flow', max_subplots=3):
        import matplotlib.pyplot as plt

        inputs, labels = self.example()
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                    label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [min]')

  
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
  
    def train(self):
        return self.make_dataset(self.train_df)

    def val(self):
        return self.make_dataset(self.val_df)

    def test(self):
        return self.make_dataset(self.test_df)

    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train()))
            # And cache it for next time
            self._example = result
        return result
    
window = WindowGenerator(input_width=168, label_width=168, shift=168, label_columns=['flow', 'occ'])
window

Total window size: 336
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [23]:
# Create an LSTM model
def create_model(input_shape):
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout

    model = Sequential([
        LSTM(32, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32, return_sequences=True),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['mean_absolute_error']
    )

    return model

# Create the model for hourly predictions for one week
model = create_model(input_shape=(window.label_width, len(window.train_df.columns)))
model.summary()
print(model.input_shape)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 168, 32)           5888      
                                                                 
 dropout_2 (Dropout)         (None, 168, 32)           0         
                                                                 
 lstm_7 (LSTM)               (None, 168, 32)           8320      
                                                                 
 dropout_3 (Dropout)         (None, 168, 32)           0         
                                                                 
 dense_4 (Dense)             (None, 168, 1)            33        
                                                                 
Total params: 14241 (55.63 KB)
Trainable params: 14241 (55.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
(None, 168, 13)


In [24]:
# Train the model
def train_model(model, window, epochs=10):
    from keras.callbacks import EarlyStopping

    early_stop = EarlyStopping(monitor='val_loss', patience=10)

    # Print model inputs and outputs
    print(f'Inputs: {model.input_shape}')
    print(f'Outputs: {model.output_shape}')

    history = model.fit(
        window.train(),
        epochs=epochs,
        validation_data=window.val(),
        callbacks=[early_stop],
        batch_size=168
    )

    return history

history = train_model(model, window)

Inputs: (None, 168, 13)
Outputs: (None, 168, 1)
Epoch 1/10
   727/165050 [..............................] - ETA: 1:32:30 - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: 